# Router Engine

Given a query, the router will pick what to run.  This shows dynamic query understanding.

## Setup

In [1]:
import os
import dotenv

dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)


True

In [1]:
# this is required since Juypyter notebook is not async by default.
import nest_asyncio
nest_asyncio.apply()

## Load data

In [2]:
from llama_index.core import SimpleDirectoryReader

lightning_ai="/teamspace/studios/this_studio/retrieval-augmented-generation/data/transcript.pdf"
github_codespace="./data/transcript.pdf"

docs = lightning_ai
documents = SimpleDirectoryReader(input_files=[docs]).load_data()


## Split document into nodes

In [3]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

## Define LLM and Embedding Model

In [4]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Indexes

- Summary Index 
- Vector Index 

Think of the index as a set of metadata over the data.  Different indexes have different index behavior.  A vector index indexes node through text embeddings.  Querying a vector index in llamaindex will return the most similar nodes. A summary index is also a simple index, and a query to it will return all of the nodes that are currently in the index.

In [5]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Turn the indexes into query engines

Each query engine represents an interface over the data that has been stored in the index.  It combines retrieval with LLM synthesis.  Each engine is specific for a certain kind of question.  This is where the use case of the router becomes significant.  

In [6]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

vector_query_engine = vector_index.as_query_engine()

## Create query tools

The query tools are simply the query engine with additional metadata.  The metadata reflects the kind of information the tool can answer.  For example, asking it to "summarize" the article would launch the summary_tool.

In [7]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the document."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the document."
    ),
)

## Define Router

Llamaindex provides several diffent selectors.  

- The LLM selectors use the LLM to output a JSON that is parsed, and corresponding indexes are queried.
- The Pydantic selects use the OpenAI Function Calling API to produce pydantic selection objects rather than raw JSON.

In this example we are using the LLMSingleSelector. The RouterQueryEngine takes in a selector type as well as a set of tools.

In [8]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    # allows us to view the intermediate steps of the query engine
    verbose=True,
)

In [9]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to the document..
The document discusses the importance of bridging the Product Company Gap, which involves moving beyond just product Market fit to build a successful company. It emphasizes the need to focus on a minimum viable segment, streamline time to value for customers, and play well in the ecosystem through partnerships. The concept of slip - simple to install, low initial cost, instant and ongoing value, and playing well in the ecosystem - is highlighted as a framework for product development. Additionally, the importance of pricing strategies, partnerships, and business models from the early stages of a startup is emphasized for long-term success.


In [10]:
print(len(response.source_nodes))

37


In [11]:
response = query_engine.query("Explain minimum viable segment.")
print(str(response))

Selecting query engine 1: The explanation of minimum viable segment would require retrieving specific context from the document to understand the concept fully..
A minimum viable segment is a small and specific market segment that a product or service is tailored for. It is a targeted group of customers with a particular problem that the product aims to solve. The focus is on dominating this niche market rather than trying to appeal to a broad audience. By successfully addressing the needs of this minimum viable segment, a business can prove its concept and viability before scaling up.


In [13]:
response = query_engine.query("How do you build a product according to this document?")
print(str(response))

Selecting query engine 1: Useful for retrieving specific context from the document..
To build a product according to this document, you should start by designing a product for go-to-market fit, ensuring that you create a minimum viable product (MVP) that addresses a valuable problem. It is crucial to triple check the value proposition before investing in hiring engineers or spending money to ensure that the product solves a problem that customers are willing to pay for. Additionally, it is important to design the product with scalability in mind, considering factors like go-to-market strategy, pricing, and building a business model that supports scaling the product into a successful company.


## Let's put everything together

Code from the above cells, have been put into the utils helper method.  Builds the query engine with both vector and summary search.

In [2]:
from llama_utils import get_router_query_engine

lightning_ai="/teamspace/studios/this_studio/retrieval-augmented-generation/data/transcript.pdf"
github_codespace="./data/transcript.pdf"
docs = lightning_ai

query_engine = get_router_query_engine(docs)

In [3]:
response = query_engine.query("Please summarize the main point of the document.")
print(str(response))

Selecting query engine 0: This choice is relevant for summarization questions related to MetaGPT, which aligns with the request to summarize the main point of the document..
The main point of the document is to discuss the importance of bridging the Product Company Gap, which involves transitioning from just having a product idea to building a product that can scale into a successful company. It emphasizes the significance of considering factors like product-market fit, go-to-market strategies, pricing, partnerships, and ecosystem integration from the early stages of product development to ensure long-term success and growth.


In [14]:
response = query_engine.query("Please summarize the secondary point of the document.")
print(str(response))

Selecting query engine 0: The document is focused on summarization questions related to MetaGPT, indicating that the secondary point may be related to this aspect..
The secondary point of the document is to emphasize the importance of bridging the Product Company Gap, which involves transitioning from just having a product idea to building a product that can scale into a successful company. This transition requires considerations such as creating a value proposition, identifying a minimum viable segment, designing for product-led growth, and forming strategic partnerships to accelerate business growth.


In [15]:
response = query_engine.query("Please summarize product led growth.")
print(str(response))

Selecting query engine 0: MetaGPT can be used for summarization tasks, making it useful for summarizing concepts like product led growth..
Product-led growth is a strategy where the product itself drives customer acquisition, retention, and expansion. It involves offering a product that is easy to install, provides instant and ongoing value, and plays well within the ecosystem. This approach focuses on creating a product that is simple to use, offers low initial cost, delivers immediate value, and integrates effectively with other platforms or partners. By prioritizing the product's ability to attract and retain users, product-led growth aims to drive business growth through the product's inherent value and user experience.


In [16]:
response = query_engine.query("Does the paper provide specific coding example?")
print(str(response))

Selecting query engine 1: The paper provides specific context from the MetaGPT paper, which may include coding examples..
The paper does not provide specific coding examples.
